In [1]:
from statistics import mean, stdev
import numpy as np
from typing import Any, Dict, Pattern, Set, Union, List
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorboard.plugins.hparams import api as hp
from tensorflow import keras
import yfinance as yf

2023-01-18 21:34:01.466265: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-18 21:34:01.674438: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-18 21:34:01.680061: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-18 21:34:01.680076: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
model = keras.models.load_model('model')

2023-01-18 21:34:11.426545: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-18 21:34:11.426568: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-18 21:34:11.426584: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (nba-176-w1a-pl): /proc/driver/nvidia/version does not exist
2023-01-18 21:34:11.426807: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:

def prepare_prediction_data(open, close, high, low, volume, week_volume, month_volume, year_volume, week_close, month_close, year_close, close_day_before, close_week_before, close_month_before, close_year_before):
    '''
    POPRZEDNI DZIEŃ:
    - cena otwarcia
    - cena zamknięcia
    - najwyższa cena
    - najniższa cena
    - ilość transakcji
    - OSTATNI tydzień:
    - cena zamknięcia
    - ilość transakcji
    - OSTATNI MIESIĄC:
    - cena zamknięcia
    - ilość transakcji
    - OSTATNI ROK:
    - cena zamknięcia
    - ilość transakcji
    '''
    df_new = pd.DataFrame()
    # 6 oryginalnych cech
    df_new['open'] = close
    df_new['open_1'] = open
    df_new['close_1'] = close
    df_new['high_1'] = high
    df_new['low_1'] = low
    df_new['volume_1'] = volume
    # 31 wygenerowanych cech
    # Średnie ceny
    df_new['avg_price_5'] = np.mean(week_close)
    df_new['avg_price_30'] = mean(month_close)
    df_new['avg_price_365'] = mean(year_close)
    df_new['ratio_avg_price_5_30'] = df_new['avg_price_5'] / df_new['avg_price_30']
    df_new['ratio_avg_price_5_365'] = df_new['avg_price_5'] / df_new['avg_price_365']
    df_new['ratio_avg_price_30_365'] = df_new['avg_price_30'] / df_new['avg_price_365']
    # Średnie woluminy
    df_new['avg_volume_5'] = mean(week_volume)
    df_new['avg_volume_30'] = mean(month_volume)
    df_new['avg_volume_365'] = mean(year_volume)
    df_new['ratio_avg_volume_5_30'] = df_new['avg_volume_5'] / df_new['avg_volume_30']
    df_new['ratio_avg_volume_5_365'] = df_new['avg_volume_5'] / df_new['avg_volume_365']
    df_new['ratio_avg_volume_30_365'] = df_new['avg_volume_30'] / df_new['avg_volume_365']
    # Odchylenia standardowe cen
    df_new['std_price_5'] = stdev(week_close)
    df_new['std_price_30'] = stdev(month_close)
    df_new['std_price_365'] = stdev(year_close)
    df_new['ratio_std_price_5_30'] = df_new['std_price_5'] / df_new['std_price_30']
    df_new['ratio_std_price_5_365'] = df_new['std_price_5'] / df_new['std_price_365']
    df_new['ratio_std_price_30_365'] = df_new['std_price_30'] / df_new['std_price_365']
    # Odchylenia standardowe woluminów
    df_new['std_volume_5'] = stdev(week_volume)
    df_new['std_volume_30'] = stdev(month_volume)
    df_new['std_volume_365'] = stdev(year_volume)
    df_new['ratio_std_volume_5_30'] = df_new['std_volume_5'] / df_new['std_volume_30']
    df_new['ratio_std_volume_5_365'] = df_new['std_volume_5'] / df_new['std_volume_365']
    df_new['ratio_std_volume_30_365'] = df_new['std_volume_30'] / df_new['std_volume_365']
    # Zwroty
    df_new['return_1'] = (close - close_day_before) / close_day_before
    df_new['return_5'] = (close - close_week_before) / close_week_before
    df_new['return_30'] = (close - close_month_before) / close_month_before
    df_new['return_365'] = (close - close_year_before) / close_year_before

    week_close_return = [((x - week_close[index-1]) / week_close[index-1]) if index-1 >= 0 else 0 for index, x in enumerate(week_close)]
    week_close_return_fix = [week_close_return[index+1] if week_close_return[index] == 0 else week_close_return[index] for index, x in enumerate(week_close_return)]
    df_new['moving_avg_5'] = mean(week_close_return_fix)

    month_close_return = [((x - month_close[index-1]) / month_close[index-1]) if index-1 >= 0 else 0 for index, x in enumerate(month_close)]
    month_close_return_fix = [month_close_return[index+1] if month_close_return[index] == 0 else month_close_return[index] for index, x in enumerate(month_close_return)]
    df_new['moving_avg_30'] = mean(month_close_return_fix)

    year_close_return = [((x - year_close[index-1]) / year_close[index-1]) if index-1 >= 0 else 0 for index, x in enumerate(year_close)]
    year_close_return_fix = [year_close_return[index+1] if year_close_return[index] == 0 else year_close_return[index] for index, x in enumerate(year_close_return)]
    df_new['moving_avg_365'] = mean(year_close_return_fix)
    # Wartości docelowe
    df_new['close'] = close
    print(df_new['avg_price_30'])
    print(type(open))
    return df_new

In [ ]:
gspc = yf.Ticker("^GSPC")
data_raw = gspc.history(period="1y")
data_raw = data_raw[["Open", "High", "Low", "Close", "Volume"]]
data_raw

,Open,High,Low,Close,Volume
Date,,,,,
2022-01-18 00:00:00-05:00,4632.240234,4632.240234,4568.700195,4577.109863,4748700000
2022-01-19 00:00:00-05:00,4588.029785,4611.549805,4530.200195,4532.759766,4465740000
2022-01-20 00:00:00-05:00,4547.350098,4602.109863,4477.950195,4482.729980,4640870000
2022-01-21 00:00:00-05:00,4471.379883,4494.520020,4395.339844,4397.939941,5589100000
2022-01-24 00:00:00-05:00,4356.319824,4417.350098,4222.620117,4410.129883,6928110000
...,...,...,...,...,...
2023-01-11 00:00:00-05:00,3932.350098,3970.070068,3928.540039,3969.610107,4303360000
2023-01-12 00:00:00-05:00,3977.570068,3997.760010,3937.560059,3983.169922,4440260000
2023-01-13 00:00:00-05:00,3960.600098,4003.949951,3947.669922,3999.090088,3939700000


In [ ]:
data_raw["Close"].iloc[-5:].tolist()

[3969.610107421875,
 3983.169921875,
 3999.090087890625,
 3990.969970703125,
 3947.659912109375]

In [ ]:
open = data_raw["Open"].iloc[-1].item()
close = data_raw["Close"].iloc[-1].item()
high = data_raw["High"].iloc[-1].item()
low = data_raw["Low"].iloc[-1].item()

volume = data_raw["Volume"].iloc[-1].tolist()
week_volume = data_raw["Volume"].iloc[-5:].tolist()
month_volume = data_raw["Volume"].iloc[-21:].tolist()
year_volume = data_raw["Volume"].iloc[-252:].tolist()

week_close = data_raw["Close"].iloc[-5:].tolist()
month_close = data_raw["Close"].iloc[-252:].tolist()
year_close = data_raw["Close"].iloc[-252:].tolist()

close_day_before = data_raw["Close"].iloc[-2].item()
close_week_before = data_raw["Close"].iloc[-5].item()
close_month_before = data_raw["Close"].iloc[-21].item()
close_year_before = data_raw["Close"].iloc[-252].item()

In [ ]:
prepare_prediction_data(open, close, high, low, volume, week_volume, month_volume, year_volume, week_close, month_close, year_close, close_day_before, close_week_before, close_month_before, close_year_before)

Series([], Name: avg_price_30, dtype: float64)
<class 'float'>


,open,open_1,close_1,high_1,low_1,volume_1,avg_price_5,avg_price_30,avg_price_365,ratio_avg_price_5_30,...,ratio_std_volume_5_365,ratio_std_volume_30_365,return_1,return_5,return_30,return_365,moving_avg_5,moving_avg_30,moving_avg_365,close


In [ ]:
from statistics import mean
lsita = [1,6,1,8,5]
lista2= [((x - lsita[index-1]) / lsita[index-1]) if index-1 >= 0 else 0 for index, x in enumerate(lsita)]
[lista2[index+1] if lista2[index] == 0 else lista2[index] for index, x in enumerate(lista2)]
# for index, x in enumerate(lsita):
#     print((lsita[index-1]))


[5.0, 5.0, -0.8333333333333334, 7.0, -0.375]

In [ ]:
mean([1])

1